In [1]:

from flask import Flask, request, jsonify, send_from_directory
import os

In [2]:
# go through
IMAGE_DIRECTORY = "./testImages"

image_files = os.listdir(IMAGE_DIRECTORY)
image_urls = [f"http://localhost:8000/testImages/{filename}" for filename in image_files]
if 'http://localhost:8000/testImages/.DS_Store' in image_urls:
        image_urls.remove('http://localhost:8000/testImages/.DS_Store')
captions = ['Expected value diagram','Cute areeba photo','weird friend graph thing','younger areeba photo']
imageCaptionDatabase = {}
for i in range(len(image_urls)):
        package = {}
        package['url'] = image_urls[i]
        package['caption'] = captions[i]
        imageCaptionDatabase[i] = package

imageCaptionDatabase
# save the imageCaptionDatabase to a json file
import json


In [3]:
# load the imageCaptionDatabase from the json file
with open('imageCaptionDatabase.json', 'r') as f:
    imageCaptionDatabase = json.load(f)
    

In [4]:
def angleFixer(H = 0,Q = 0):
    if(H>45):
        H_o = 90-2*(H-45)
    if(Q>90):
        Q_o = Q-90
        H_o = Q_o-H_o

    print('QWP:',Q_o,'\nHWP:',H_o)
angleFixer(Q=120,H=70)

QWP: 30 
HWP: -10


# Telegram

In [5]:
from dotenv import load_dotenv
load_dotenv()

# Access the API key
bot_token = os.getenv('TELEGRAM-KEY')

# we need to make a basic tele

In [6]:
import requests

def set_webhook(bot_token, webhook_url):
    url = f'https://api.telegram.org/bot{bot_token}/setWebhook'
    payload = {
        'url': webhook_url
    }
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        print('Webhook set successfully!')
    else:
        print('Failed to set webhook.')
        print('Error:', response.text)


webhook_url = 'webhook.azalamer.com'
# this needs us to use ngrok, note that we need to find a way to keep this constant, 
# or push it to my domain url as a subaddress

set_webhook(bot_token, webhook_url+'/telegram-webhook')

Webhook set successfully!


In [7]:
import requests

def get_webhook_info(bot_token):
    base_url = f"https://api.telegram.org/bot{bot_token}/"
    endpoint = "getWebhookInfo"
    
    response = requests.get(base_url + endpoint)
    
    if response.status_code == 200:
        webhook_info = response.json()
        return webhook_info['result']
    else:
        return f"Error: {response.status_code} - {response.text}"

# Replace 'YOUR_BOT_TOKEN' with your actual bot token

webhook_info = get_webhook_info(bot_token)
print("Current Webhook Info:")
print(webhook_info)

Current Webhook Info:
{'url': 'webhook.azalamer.com/telegram-webhook', 'has_custom_certificate': False, 'pending_update_count': 2, 'last_error_date': 1720502992, 'last_error_message': 'Wrong response from the webhook: 502 Bad Gateway', 'max_connections': 40, 'ip_address': '188.114.97.0'}


In [8]:
from io import BytesIO
from PIL import Image
SAVE_DIRECTORY = "./images"
BASE_URL = f"https://api.telegram.org/bot{bot_token}/"
if not os.path.exists(SAVE_DIRECTORY):
    os.makedirs(SAVE_DIRECTORY)

def get_file_path(file_id):
    url = f"{BASE_URL}getFile"
    response = requests.get(url, params={"file_id": file_id})
    return response.json()["result"]["file_path"]

def download_file(file_path):
    url = f"https://api.telegram.org/file/bot{bot_token}/{file_path}"
    response = requests.get(url)
    return BytesIO(response.content)

def process_image(image_data):
    # Example processing: resize image
    return image_data
def databaseAdder(file_id,caption,url):
    # open the file
    with open('imageCaptionDatabase.json', 'r') as f:
        imageCaptionDatabase = json.load(f)
    imageCaptionDatabase[len(imageCaptionDatabase)] = {'url':url,'caption':caption,'file_id':file_id}
    with open('imageCaptionDatabase.json', 'w') as f:
        json.dump(imageCaptionDatabase, f)
        
def renumber_dict_keys(data, n):
    # Convert string keys to integers for sorting
    sorted_keys = sorted(map(int, data.keys()))
    
    # Remove the first n items
    keys_to_remove = sorted_keys[:n]
    for key in keys_to_remove:
        del data[str(key)]
    
    # Create a new dictionary with renumbered keys
    new_data = {}
    for i, key in enumerate(sorted_keys[n:]):
        new_key = str(i)
        new_data[new_key] = data[str(key)]
    
    return new_data

In [9]:
from flask_cors import CORS

app = Flask(__name__)
CORS(app)


IMAGE_DIRECTORY = "./images"

@app.route('/telegram-webhook', methods=['POST'])
# load the imageCaptionDatabase from the json file

def webhook():
    # Get the input data from the request
    input_data = request.json

    # Extract the relevant information from the input data
    
    try:
        file = input_data['message']['photo'][-1] # Get the largest photo
        file_id = file['file_id']  
        caption = ''
        try:
            caption = input_data['message']['caption']
        except Exception as e:
            print('error',e)
        print({'caption':caption,'file_id':file_id})
        file_path = get_file_path(file_id)
        # at this point, we need to add it to our imageCaptionDatabase. The database goes 'url':url,'caption':caption, 'file_id':file_id. each time we get a new image, we add it as the next index in the database
        databaseAdder(file_id,caption,webhook_url+SAVE_DIRECTORY[1:]+'/'+file_id+'.jpg')
        image_data = Image.open(download_file(file_path))
        
        
        # processed_image = process_image(image_data)
        
        # Save the processed image
        save_path = os.path.join(SAVE_DIRECTORY, f"{file_id}.jpg")
        image_data.save(save_path)
    except Exception as e:
        print('error',e)
        print('No image found in the message')
        messange_text = input_data['message']['text']

        if(messange_text.lower() == 'wipe'):
            with open('imageCaptionDatabase.json', 'w') as f:
                json.dump({}, f)
            # delete all the images in the images folder
            for filename in os.listdir(SAVE_DIRECTORY):
                file_path = os.path.join(SAVE_DIRECTORY, filename)
                os.remove(file_path)
            with open('imageCaptionDatabase.json', 'r') as f:
                imageCaptionDatabase = json.load(f)
            response = f'{len(imageCaptionDatabase)} images and captions were deleted.'

        
        elif(messange_text.lower() == 'status'):
            with open('imageCaptionDatabase.json', 'r') as f:
                imageCaptionDatabase = json.load(f)
            response = f"Number of images: {len(imageCaptionDatabase)}"
        else:
            response = 'No image or caption was sent'

        chat_id = input_data['message']['chat']['id']
        result = response
        # TODO we should add a "wipe" command to the bot, and a "status" command to the bot as well
        print(f"message_text: {response}")

        # Send the result back to Telegram
        send_message(chat_id, result)



    return 'OK'
def send_message(chat_id, text):
    # Send a message back to Telegram using the sendMessage method
    url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    payload = {
        'chat_id': chat_id,
        'text': text
    }
    requests.post(url, json=payload)




@app.route('/', methods=['GET'])
def serve_index():
    return send_from_directory('.', 'index.html')

@app.route('/controller', methods=['POST'])
# on this controller route, we will take a request which contains the number of images requested, so that the user only gets the number of images they want
# then earlier we constructed our imageDatabase which now includes captions, so we will return the imageDatabase to the user

def handle_request():
    # the request should contain the number of images requested
    data = request.json
    num_images = data['numImages']
    # remove .DS_Store file from list of images
    responsePackage = {}
    with open('imageCaptionDatabase.json', 'r') as f:
        imageCaptionDatabase = json.load(f)

    if(num_images > len(imageCaptionDatabase)):
        num_images = len(imageCaptionDatabase)
    # this makes it so we can only accept the number of images we have to give
    for i in range(num_images):
        responsePackage[i] = imageCaptionDatabase[str(i)]
    

    response_data = {
        'success': True,
        'message': 'Image URLs retrieved successfully',
        'imageUrls': imageCaptionDatabase,
        'numImages': num_images
    }

    return jsonify(response_data)


@app.route('/images/<path:filename>', methods=['GET'])
def serve_image(filename):
    return send_from_directory(IMAGE_DIRECTORY, filename)


@app.route('/confirmation',methods = ['POST'])
def conformationAndDeletion():
    data = request.json['imageIDs']
    # data is just a list of the fileID's that we need to delete
    # we should delete the image from the images folder
    for i in range(len(data)):
        save_path = os.path.join(SAVE_DIRECTORY, f"{data[i]}.jpg")
        os.remove(save_path)
    # we should also delete the image from the imageCaptionDatabase
    with open('imageCaptionDatabase.json', 'r') as f:
        imageCaptionDatabase = json.load(f)
    with open('imageCaptionDatabase.json', 'w') as f:
        json.dump(renumber_dict_keys(imageCaptionDatabase,len(data)), f)
    
    # find the index in the database that has the fileID that we need to delete
    
    # upon reflection, while this is implicit, the only images that get served are the ones at the top, not the back. So if I want I can just delete the first n entries in the database and renumber

    

    return jsonify({'success':True,'message':'Image deleted successfully'})

if __name__ == '__main__':
    print('Starting the Flask app')
    print(f"bot token: {bot_token[:5]}...{bot_token[-5:]}")
    app.run(port=8000)


Starting the Flask app
bot token: 65579...mrW8k
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit


error 'photo'
No image found in the message
message_text: Number of images: 0


127.0.0.1 - - [09/Jul/2024 00:29:56] "POST /telegram-webhook HTTP/1.1" 200 -


error 'photo'
No image found in the message
message_text: Number of images: 0


127.0.0.1 - - [09/Jul/2024 00:29:59] "POST /telegram-webhook HTTP/1.1" 200 -


error 'photo'
No image found in the message
message_text: No image or caption was sent


127.0.0.1 - - [09/Jul/2024 00:30:08] "POST /telegram-webhook HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 00:30:18] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2024 00:30:18] "GET /favicon.ico HTTP/1.1" 404 -


error 'caption'
{'caption': '', 'file_id': 'AgACAgEAAxkBAAOLZozK9IopNZCBrJgQ9vFgnzpvCr0AAjysMRu5V2lEYaRSg2AT43ABAAMCAAN5AAM1BA'}


127.0.0.1 - - [09/Jul/2024 00:30:30] "POST /telegram-webhook HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 00:32:10] "GET /controller HTTP/1.1" 405 -
127.0.0.1 - - [09/Jul/2024 00:32:16] "GET /controller HTTP/1.1" 405 -


# Overall Connector


In [ ]:
from flask_cors import CORS

app = Flask(__name__)
CORS(app)


IMAGE_DIRECTORY = "./testImages"

@app.route('/', methods=['GET'])
def serve_index():
    return send_from_directory('.', 'index.html')

@app.route('/controller', methods=['POST'])
# on this controller route, we will take a request which contains the number of images requested, so that the user only gets the number of images they want
# then earlier we constructed our imageDatabase which now includes captions, so we will return the imageDatabase to the user

def handle_request():
    # the request should contain the number of images requested
    data = request.json
    num_images = data['numImages']
    # remove .DS_Store file from list of images
    responsePackage = {}

    if(num_images > len(imageCaptionDatabase)):
        num_images = len(imageCaptionDatabase)
    # this makes it so we can only accept the number of images we have to give
    for i in range(num_images):
        responsePackage[i] = imageCaptionDatabase[i]
    

    response_data = {
        'success': True,
        'message': 'Image URLs retrieved successfully',
        'imageUrls': imageCaptionDatabase,
        'numImages': num_images
    }

    return jsonify(response_data)

@app.route('/testImages/<path:filename>', methods=['GET'])
def serve_image(filename):
    return send_from_directory(IMAGE_DIRECTORY, filename)

if __name__ == '__main__':
    app.run(port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [07/Jul/2024 18:56:16] "OPTIONS /controller HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2024 18:56:16] "POST /controller HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2024 18:56:16] "GET /testImages/PXL_20240705_220718077.jpg HTTP/1.1" 304 -
127.0.0.1 - - [07/Jul/2024 18:56:16] "GET /testImages/IMG-20240701-WA0005.jpg HTTP/1.1" 304 -
127.0.0.1 - - [07/Jul/2024 18:59:01] "OPTIONS /controller HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2024 18:59:01] "POST /controller HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2024 18:59:01] "GET /testImages/PXL_20240705_220718077.jpg HTTP/1.1" 304 -
127.0.0.1 - - [07/Jul/2024 18:59:01] "GET /testImages/IMG-20240701-WA0005.jpg HTTP/1.1" 304 -
127.0.0.1 - - [07/Jul/2024 18:59:42] "OPTIONS /controller HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2024 18:59:42] "POST /controller HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2024 18:59:42] "GET /testImages/PXL_20240705_220718077.jpg HTTP/1.1" 304 -
127.0.0.1 - - [07/Jul/2024 18